In [ ]:
%load_ext autoreload
%autoreload 2
import SBGeom
import numpy as np
import jax
import jax.numpy as jnp
import meshio

### Simple Flux Surface Meshing

In [ ]:

vmec_file = "../example_data/helias5_vmec.nc4"
s1 = SBGeom.Flux_Surfaces_From_HDF5(vmec_file)
s1.Mesh_Surface(1.0,0.0,300,500,- 0.00 * np.pi, 2 * np.pi / 5).write("LCFS.vtk")



### Plotting Tiled Surfaces


In [ ]:
tiled = True
if tiled:
    snorm = SBGeom.Flux_Surfaces_Normal_Extended(s1)
    mtile = snorm.Mesh_Tiled_Surface(1.0,0.4, 200,50,0.9, 0.7*  np.pi, 2.3 * np.pi)

    SBGeom.to_meshio(mtile).write("tiled_fw.stl")

    edge = snorm.Mesh_Surfaces_Closed(1.0,1.0,0.415,1.4,400,400, 0.8 * np.pi , 2.2 * np.pi)
    SBGeom.to_meshio(edge).write("blanket.stl")


    mtile = snorm.Mesh_Tiled_Surface(1.0,1.4, 100,20,0.9, 0.9*  np.pi, 2.1 * np.pi)

    SBGeom.to_meshio(mtile).write("tiled_blanket.stl")


### Plotting VTK files with B strength

In [ ]:
import h5py
import meshio
def to_vtk_plus_b(filename, Nu, Nv, output):
    with h5py.File(filename,"r") as f:
        bmnc = np.array(f['bmnc'] )
        mpol = np.array(f['xm_nyq'])
        ntor = np.array(f['xn_nyq'])

    u = np.linspace(0, 2 * np.pi , Nu, endpoint=False)
    v = np.linspace(0, 2 * np.pi , Nv, endpoint=False)
    uu, vv= np.meshgrid(u,v)
    uu = uu.T
    vv = vv.T


    bmag = np.sum(bmnc[-1,:][:,np.newaxis] * np.cos(mpol[:,np.newaxis] * uu.ravel()[np.newaxis, : ] - ntor[:,np.newaxis] * vv.ravel()[np.newaxis,:]),axis=0)
    s1 = SBGeom.Flux_Surfaces_From_HDF5(filename)
    mesh = SBGeom.to_meshio(s1.Mesh_Surface(1.0,0.0,Nv,Nu, 0.0, 2* np.pi))
    mesh.point_data = {"bm" : bmag }
    mesh.write(output+".vtk")

#to_vtk_plus_b(vmec_file,800,800,"bstrength")

### Plotting VTK lines with a grid of poloidal and toroidal lines

In [ ]:
import itertools
def grid_to_vtk(filename,Nu,Nv,output, linedec = 300):
    s1 = SBGeom.Flux_Surfaces_From_HDF5(filename)
    u_lines = np.linspace(0.0, 2 * np.pi , linedec, endpoint=False)
    v_lines = np.linspace(0.0, 2 * np.pi , linedec, endpoint=False)

    v_linedec = np.linspace(0.0, 2 * np.pi, Nv, endpoint = True)
    u_linedec = np.linspace(0.0, 2 * np.pi, Nu, endpoint = True)
    

    ulines = np.hstack([s1.Return_Position(np.ones(u_lines.shape), np.zeros(u_lines.shape), u_lines, np.ones(u_lines.shape) * v_linedec[i]).T for i in range(Nv)]).T
    vlines = np.hstack([s1.Return_Position(np.ones(u_lines.shape), np.zeros(u_lines.shape),np.ones(v_lines.shape)* u_linedec[i], v_lines).T for i in range(Nu)]).T
    
    ulist = list(itertools.chain.from_iterable([[[i + j * linedec, (i+1)%linedec + j * linedec] for i in range(linedec)] for j in range(Nv)]))    
    vlist = list(itertools.chain.from_iterable([[[i + j * linedec, (i+1)%linedec + j * linedec] for i in range(linedec)] for j in range(Nu)]))    
    meshio.Mesh(points = ulines, cells = [("line", ulist )]).write("ulines_"+output+".vtk")
    meshio.Mesh(points = vlines, cells = [("line", vlist )]).write("vlines_"+output+".vtk")
    

grid_to_vtk(vmec_file,20,60, "grid")


### Creating Extension Files using reduced boundary representation

In [ ]:

b1 = SBGeom.Optimise_Blanket.Stellarator_Blanket_Optimizer(vmec_file)

In [ ]:
b1.flux_surfaces.Mesh_Surface(1.0,1.39, 60,100,0.0, 0.5 * np.pi).write("blanket_normal_vec.vtk")
d_list = [1.4]
Rmnc, Zmns = b1.Optimize_d_list([lambda u,v,fs : np.ones(u.shape) * d for d in d_list],30,50)

In [ ]:
b2 = SBGeom.Flux_Surfaces_Fourier_Extended(s1, d_list, Rmnc, Zmns )
b2.Mesh_Surface(1.0,1.39,60,100,0.5 * np.pi ,np.pi).write("blanket_fit.vtk")

In [ ]:
npl = 10
nbl = 10
slist = np.linspace(0.0,1.0,npl)
dlist = np.linspace(0.0,0.0, npl)
dbl = np.linspace(0.0, 1.39, nbl)
sbl = np.linspace(1.0,1.0, nbl)
stot = np.concatenate([slist,sbl])
dtot = np.concatenate([dlist,dbl])
b2.Mesh_Tetrahedrons(stot,dtot, 100,100, np.pi, np.pi * 1.5).write("tetrahedral_mesh.vtk")

In [ ]:
mtile = b2.Mesh_Tiled_Surface(1.0,1.3999, 100,20,0.9, 0.8*  np.pi, 2.2 * np.pi)

SBGeom.to_meshio(mtile).write("tiled_blanket.stl")

### Using the Direct Approach


In [ ]:

b1 = SBGeom.Optimise_Blanket_Direct.Stellarator_Blanket_Optimizer_Direct(vmec_file)
d_list = [1.4]
Rmnc, Zmns = b1.Optimize_d_list([lambda u,v,fs : np.ones(u.shape) * d for d in d_list], 30, 50)

In [ ]:
direct_approach = SBGeom.Flux_Surfaces_Fourier_Extended(s1, d_list, Rmnc, Zmns)
direct_approach.Mesh_Surface(1.0,1.39,60,100,0.5 * np.pi ,np.pi).write("blanket_fit2.vtk")

In [ ]:
npl = 10
nbl = 10
slist = np.linspace(0.0,1.0,npl)
dlist = np.linspace(0.0,0.0, npl)
dbl = np.linspace(0.0, 1.39, nbl)
sbl = np.linspace(1.0,1.0, nbl)
stot = np.concatenate([slist,sbl])
dtot = np.concatenate([dlist,dbl])
direct_approach.Mesh_Tetrahedrons(stot,dtot, 100,100, np.pi, np.pi * 1.5).write("tetrahedral_mesh2.vtk")

### Non-uniform thickness wall fitting

In [ ]:
b1 = SBGeom.Optimise_Blanket_Direct.Stellarator_Blanket_Optimizer_Direct(vmec_file)

def dfunc(u,v, fs):

    return 1.4* np.ones(u.shape) + 0.8 * np.sin(u + 0.5 * np.pi)

rmnc, zmns = b1.Optimize_d_list([dfunc], 30,50, optimize_arclength=True)
direct_approach = SBGeom.Flux_Surfaces_Fourier_Extended(s1, [1.0], rmnc, zmns) # this d here is not physical distance but just a label
direct_approach.Mesh_Surface(1.0,0.99,60,100,0.0 * np.pi ,np.pi / 5).write("blanket_fit_varying.vtk")
npl = 10
nbl = 10
slist = np.linspace(0.0,1.0,npl)
dlist = np.linspace(0.0,0.0, npl)
dbl = np.linspace(0.0, 0.99, nbl)
sbl = np.linspace(1.0,1.0, nbl)
stot = np.concatenate([slist,sbl])
dtot = np.concatenate([dlist,dbl])
direct_approach.Mesh_Tetrahedrons(stot,dtot, 100,100, 0.0, np.pi * 1.5).write("tetrahedral_mesh_varying.vtk")